<a href="https://colab.research.google.com/github/mks999/DML_Economics/blob/main/Main_DML_PLR_and_Feature_Inference_PanelMultiDiD.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install doubleml scikit-learn pandas numpy openpyxl


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 443.3/443.3 kB 8.7 MB/s eta 0:00:00


In [2]:
import pandas as pd

file_path = '/content/Final Imputed dataset with dummies 7.7.25.xlsx'
df = pd.read_excel(file_path)

print(df.columns)
print(df.head())


Index(['country', 'time', 'mhtx', 'rnd', 'tariff', 'mobile', 'uv_index',
       'gfcf', 'fdi', 'lnwages', 'reer', 'd1', 'aggemp', 'ctryid',
       'dmy_uvindex', 'dmy_mhtx', 'msch', 'internet', 'covid_dummy',
       'L_aggemp'],
      dtype='object')
     country  time     mhtx       rnd  tariff    mobile  uv_index     gfcf  \
0  Argentina  1990  23.6244  0.472133  14.550  0.036767      40.5  13.9970   
1  Argentina  1991  23.6244  0.472133  12.383  0.075516      50.2  14.6370   
2  Argentina  1992  23.6244  0.450528  12.660  0.138792      52.4  16.7024   
3  Argentina  1993  28.9722  0.441513  12.530  0.329148      52.9  19.6881   
4  Argentina  1994  30.0064  0.436208  11.723  0.699252      57.3  19.9656   

       fdi  lnwages     reer  d1  aggemp  ctryid  dmy_uvindex  dmy_mhtx  \
0  1.29888  22.4846  195.851   0    46.6       1          0.0       0.0   
1  1.28558  22.7368  193.781   0    47.0       1          0.0       0.0   
2  1.93679  22.6555  188.565   0    47.3       1       

In [3]:
# ===============================
# STEP 2: Define Variables
# ===============================

# Panel entity and time variables
id_col = 'ctryid'      # Country identifier
time_col = 'time'       # Time/year variable

# Outcome variable
y_col = 'aggemp'        # Aggregate employment

# Treatment variable (for staggered DiD)
treatment_col = 'mhtx'  # Main treatment variable

# Control variables (as per your dataset)
x_cols = [
    'gfcf',       # Gross fixed capital formation
    'rnd',        # R&D
    'internet',   # Internet penetration
    'mobile',     # Mobile penetration
    'fdi',        # Foreign direct investment
    'lnwages',    # Log of wages
    'tariff',     # Tariff rate
    'msch',       # Mean years of schooling
    'reer',       # Real effective exchange rate

]


In [4]:
# ===============================
# STEP 3: Create Staggered Treatment Variable
# ===============================
import numpy as np

def get_first_treat(subdf):
    treated = subdf.loc[subdf[treatment_col] == 1, time_col]
    return treated.iloc[0] if not treated.empty else np.inf

df['first_treat'] = df.groupby(id_col).apply(get_first_treat).reindex(df.index).values

# ===============================
# STEP 4: Drop Missing Values in Required Columns
# ===============================
required_cols = [id_col, time_col, y_col, treatment_col, 'first_treat'] + x_cols
df = df.dropna(subset=required_cols)

/tmp/ipython-input-4-1521776442.py:10: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  df['first_treat'] = df.groupby(id_col).apply(get_first_treat).reindex(df.index).values


In [5]:
# STEP 6: Set Up DoubleML Panel Data
from doubleml.data import DoubleMLPanelData

dml_panel_data = DoubleMLPanelData(
    data=df,
    y_col=y_col,
    d_cols='first_treat',
    id_col=id_col,
    t_col=time_col,
    x_cols=x_cols
)


In [6]:
# STEP 7: Specify Machine Learning Models
from sklearn.ensemble import GradientBoostingRegressor, RandomForestRegressor

ml_g = GradientBoostingRegressor(n_estimators=200, max_depth=4, random_state=42)
ml_m = RandomForestRegressor(n_estimators=200, max_depth=4, random_state=42)


In [27]:
# STEP 8: Fit DoubleMLDIDMulti (Staggered DiD)
from doubleml.did import DoubleMLDIDMulti

dml_did = DoubleMLDIDMulti(
    obj_dml_data=dml_panel_data,
    ml_g=ml_g,
    ml_m=ml_m,
    control_group='never_treated'
)

dml_did.fit()
print(dml_did)

ValueError: No valid group-time combinations found. Please check the treatment group values and time period values (and anticipation).

In [8]:
#New DML_IRM INstance Run
!pip install doubleml scikit-learn pandas numpy openpyxl


In [9]:
import pandas as pd

file_path = '/content/Final Imputed dataset with dummies 7.7.25.xlsx'
df = pd.read_excel(file_path)

print(df.columns)
print(df.head())


Index(['country', 'time', 'mhtx', 'rnd', 'tariff', 'mobile', 'uv_index',
       'gfcf', 'fdi', 'lnwages', 'reer', 'd1', 'aggemp', 'ctryid',
       'dmy_uvindex', 'dmy_mhtx', 'msch', 'internet', 'covid_dummy',
       'L_aggemp'],
      dtype='object')
     country  time     mhtx       rnd  tariff    mobile  uv_index     gfcf  \
0  Argentina  1990  23.6244  0.472133  14.550  0.036767      40.5  13.9970   
1  Argentina  1991  23.6244  0.472133  12.383  0.075516      50.2  14.6370   
2  Argentina  1992  23.6244  0.450528  12.660  0.138792      52.4  16.7024   
3  Argentina  1993  28.9722  0.441513  12.530  0.329148      52.9  19.6881   
4  Argentina  1994  30.0064  0.436208  11.723  0.699252      57.3  19.9656   

       fdi  lnwages     reer  d1  aggemp  ctryid  dmy_uvindex  dmy_mhtx  \
0  1.29888  22.4846  195.851   0    46.6       1          0.0       0.0   
1  1.28558  22.7368  193.781   0    47.0       1          0.0       0.0   
2  1.93679  22.6555  188.565   0    47.3       1       

In [10]:
# Panel entity and time variables
id_col = 'country'
time_col = 'time'

# Outcome and treatment
y_col = 'aggemp'
treatment_col = 'mhtx'

# Controls (update if needed)
x_cols = [
    'gfcf', 'rnd', 'internet', 'mobile', 'fdi',
    'lnwages', 'tariff', 'msch', 'reer', 'covid_dummy'
]


In [11]:
# Drop missing values in required columns
required_cols = [y_col, treatment_col] + x_cols
df_irm = df[required_cols].dropna()


In [12]:
from doubleml import DoubleMLData

dml_data_irm = DoubleMLData(df_irm, y_col=y_col, d_cols=treatment_col, x_cols=x_cols)


In [13]:
from sklearn.ensemble import GradientBoostingRegressor, RandomForestRegressor

ml_g = GradientBoostingRegressor(n_estimators=200, max_depth=4, random_state=42)
ml_m = RandomForestRegressor(n_estimators=200, max_depth=4, random_state=42)


### DoubleML PLR with Continuous Treatment

Here's the code to implement the Partially Linear Regression (PLR) model using DoubleML, with `mhtx` as the continuous treatment variable.

In [29]:
from doubleml import DoubleMLData, DoubleMLPLR
from sklearn.ensemble import GradientBoostingRegressor, RandomForestRegressor

# Prepare your data as before
y_col = 'aggemp'
treatment_col = 'mhtx' # mhtx is a continuous treatment variable
x_cols = [
    'gfcf', 'rnd', 'internet', 'mobile', 'fdi',
    'lnwages', 'tariff', 'msch', 'reer', 'covid_dummy' # Removed 'uv_index'
]

# Drop missing values in required columns
df_plr = df[[y_col, treatment_col] + x_cols].dropna()

# Set up DoubleML Data for PLR
dml_data = DoubleMLData(df_plr, y_col=y_col, d_cols=treatment_col, x_cols=x_cols)

# Specify Machine Learning Models
ml_g = GradientBoostingRegressor(n_estimators=200, max_depth=4, random_state=42)
ml_m = RandomForestRegressor(n_estimators=200, max_depth=4, random_state=42)
ml_l = RandomForestRegressor(n_estimators=200, max_depth=4, random_state=42)

# Fit DoubleMLPLR
# Removed store_models=True from constructor
dml_plr = DoubleMLPLR(
    dml_data,
    ml_g=ml_g,
    ml_m=ml_m,
    ml_l=ml_l
)
# Added store_models=True to the fit method
dml_plr.fit(store_models=True)

# Print the summary of the results
print(dml_plr.summary)

/usr/local/lib/python3.11/dist-packages/doubleml/plm/plr.py:109: UserWarning: A learner ml_g has been provided for score = "partialling out" but will be ignored. "A learner ml_g is not required for estimation.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/sklearn/utils/deprecation.py:151: FutureWarning: 'force_all_finite' was renamed to 'ensure_all_finite' in 1.6 and will be removed in 1.8.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/sklearn/utils/deprecation.py:151: FutureWarning: 'force_all_finite' was renamed to 'ensure_all_finite' in 1.6 and will be removed in 1.8.
  warnings.warn(


          coef   std err         t     P>|t|     2.5 %    97.5 %
mhtx -0.032724  0.011889 -2.752372  0.005917 -0.056026 -0.009421


In [31]:
from doubleml import DoubleMLData, DoubleMLPLR
from sklearn.ensemble import GradientBoostingRegressor, RandomForestRegressor

# Prepare your data as before
y_col = 'aggemp'
treatment_col = 'mhtx' # mhtx is a continuous treatment variable
x_cols = [
    'gfcf', 'rnd', 'internet', 'mobile', 'fdi',
    'lnwages', 'tariff', 'msch', 'reer'
]

# Drop missing values in required columns
df_plr = df[[y_col, treatment_col] + x_cols].dropna()

# Set up DoubleML Data for PLR
dml_data = DoubleMLData(df_plr, y_col=y_col, d_cols=treatment_col, x_cols=x_cols)

# Specify Machine Learning Models
ml_g = GradientBoostingRegressor(n_estimators=200, max_depth=4, random_state=42)
ml_m = RandomForestRegressor(n_estimators=200, max_depth=4, random_state=42)
ml_l = RandomForestRegressor(n_estimators=200, max_depth=4, random_state=42) # Added ml_l

# Fit DoubleMLPLR
dml_plr = DoubleMLPLR(dml_data, ml_g=ml_g, ml_m=ml_m, ml_l=ml_l) # Added ml_l argument
# Added store_models=True to the fit method
dml_plr.fit(store_models=True)


# Print the summary of the results
print(dml_plr.summary)

/usr/local/lib/python3.11/dist-packages/doubleml/plm/plr.py:109: UserWarning: A learner ml_g has been provided for score = "partialling out" but will be ignored. "A learner ml_g is not required for estimation.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/sklearn/utils/deprecation.py:151: FutureWarning: 'force_all_finite' was renamed to 'ensure_all_finite' in 1.6 and will be removed in 1.8.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/sklearn/utils/deprecation.py:151: FutureWarning: 'force_all_finite' was renamed to 'ensure_all_finite' in 1.6 and will be removed in 1.8.
  warnings.warn(


          coef   std err        t     P>|t|     2.5 %    97.5 %
mhtx -0.034649  0.011984 -2.89132  0.003836 -0.058137 -0.011161


In [32]:
from doubleml import DoubleMLData, DoubleMLPLR
from sklearn.ensemble import GradientBoostingRegressor, RandomForestRegressor

# Prepare your data as before
y_col = 'aggemp'
treatment_col = 'mhtx' # mhtx is a continuous treatment variable
x_cols = [
    'gfcf', 'rnd', 'internet', 'mobile', 'fdi',
    'lnwages', 'tariff', 'msch', 'reer', 'covid_dummy', 'dmy_mhtx'
]

# Drop missing values in required columns
df_plr = df[[y_col, treatment_col] + x_cols].dropna()

# Set up DoubleML Data for PLR
dml_data = DoubleMLData(df_plr, y_col=y_col, d_cols=treatment_col, x_cols=x_cols)

# Specify Machine Learning Models
ml_g = GradientBoostingRegressor(n_estimators=200, max_depth=4, random_state=42)
ml_m = RandomForestRegressor(n_estimators=200, max_depth=4, random_state=42)
ml_l = RandomForestRegressor(n_estimators=200, max_depth=4, random_state=42) # Added ml_l

# Fit DoubleMLPLR
dml_plr = DoubleMLPLR(dml_data, ml_g=ml_g, ml_m=ml_m, ml_l=ml_l) # Added ml_l argument
# Added store_models=True to the fit method
dml_plr.fit(store_models=True)


# Print the summary of the results
print(dml_plr.summary)

/usr/local/lib/python3.11/dist-packages/doubleml/plm/plr.py:109: UserWarning: A learner ml_g has been provided for score = "partialling out" but will be ignored. "A learner ml_g is not required for estimation.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/sklearn/utils/deprecation.py:151: FutureWarning: 'force_all_finite' was renamed to 'ensure_all_finite' in 1.6 and will be removed in 1.8.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/sklearn/utils/deprecation.py:151: FutureWarning: 'force_all_finite' was renamed to 'ensure_all_finite' in 1.6 and will be removed in 1.8.
  warnings.warn(


          coef   std err         t     P>|t|     2.5 %    97.5 %
mhtx  0.045272  0.013092  3.457864  0.000544  0.019611  0.070933


### Inspecting Feature Importance of Control Variables

While DoubleML PLR doesn't provide coefficients for control variables in the final output, you can inspect the fitted first-stage models (`ml_g`, `ml_m`, and `ml_l`) to see the importance of each control variable in predicting the outcome and treatment.

For tree-based models like the ones used (`GradientBoostingRegressor` and `RandomForestRegressor`), feature importance is a relevant metric.

### Inspecting Feature Importance of Control Variables

Now that the `DoubleMLPLR` model has been fitted with `store_models=True`, we can access the fitted first-stage models (`ml_g`, `ml_m`, and `ml_l`) to see the importance of each control variable in predicting the outcome and treatment.

For the tree-based models used (`GradientBoostingRegressor` and `RandomForestRegressor`), feature importance is a relevant metric to understand which variables were most influential in the first-stage predictions.

### Corrected DoubleML PLR Fit and Feature Importance Display

This code block combines the initialization and fitting of the `DoubleMLPLR` model with the correct method for storing fitted models (`store_models=True` in the `.fit()` method), and then accesses and displays the feature importances of the first-stage models.

Ensure that `df_plr`, `y_col`, `treatment_col`, and `x_cols` are defined in previous cells before running this code.

### DoubleML PLR with Model Inspection and Feature Importance Extraction

This code block performs the following steps:
1.  Prepares the data, defining outcome, treatment, and control variables.
2.  Initializes and fits the `DoubleMLPLR` model, ensuring fitted models are stored using `fit(store_models=True)`.
3.  Prints the main `DoubleMLPLR` results summary.
4.  Inspects and prints the nested structure of the `dml_plr.models` attribute to understand how fitted base learners are stored.
5.  Includes a function to collect feature importances from the nested model structure across all repetitions and folds.

In [40]:
import pandas as pd
import numpy as np
from doubleml import DoubleMLData, DoubleMLPLR
from sklearn.ensemble import GradientBoostingRegressor, RandomForestRegressor

# ------------------------------------------------------------------
# 1.  PREPARE DATA  (use your own path / variable names)
# ------------------------------------------------------------------
file_path = '/content/Final Imputed dataset with dummies 7.7.25.xlsx'
df        = pd.read_excel(file_path)

y_col         = 'aggemp'
treatment_col = 'mhtx'
x_cols = [
    'uv_index', 'gfcf', 'rnd', 'internet', 'mobile', 'fdi',
    'lnwages', 'tariff', 'msch', 'reer', 'covid_dummy'
]

df_plr = df[[y_col, treatment_col] + x_cols].dropna()
dml_data = DoubleMLData(df_plr, y_col=y_col, d_cols=treatment_col, x_cols=x_cols)

# ------------------------------------------------------------------
# 2.  FIT DOUBLEML-PLR  (store fitted models)
# ------------------------------------------------------------------
ml_l = GradientBoostingRegressor(n_estimators=200, max_depth=4, random_state=42)
ml_m = RandomForestRegressor(n_estimators=200,  max_depth=4, random_state=42)

dml_plr = DoubleMLPLR(dml_data, ml_l=ml_l, ml_m=ml_m, n_folds=5, n_rep=1)
dml_plr.fit(store_models=True)          # <- critical line

print('\nDoubleMLPLR main results\n', dml_plr.summary, '\n')

# ------------------------------------------------------------------
# 3.  INSPECT THE NESTED "models" STRUCTURE
# ------------------------------------------------------------------
# Print the full models structure to diagnose the KeyError
print("Full dml_plr.models structure:")
print(dml_plr.models)
print("-" * 60)

# ------------------------------------------------------------------
# 4.  FEATURE-IMPORTANCE EXTRACTION (ALL REPS / FOLDS)
# ------------------------------------------------------------------
def collect_importances(nuisance_key):
    """Return a (rep,fold,feature) DataFrame of importances for one nuisance."""
    records = []
    # Iterate over repetitions (n_rep in dml_plr)
    for rep_idx in range(dml_plr.n_rep):
        # Access the list of fold models for this repetition and treatment variable
        # Structure is dml_plr.models[nuisance_key][treatment_col][rep_idx]
        if treatment_col not in dml_plr.models[nuisance_key]:
             print(f"Warning: Treatment column '{treatment_col}' not found for nuisance key '{nuisance_key}' in dml_plr.models structure.")
             continue

        fold_models_list = dml_plr.models[nuisance_key][treatment_col][rep_idx]

        # Iterate over folds (n_folds in dml_plr)
        for fold_idx in range(dml_plr.n_folds):
            # Access the fitted model for this fold within the repetition list
            fitted_model = fold_models_list[fold_idx]

            if hasattr(fitted_model, 'feature_importances_'):
                importances = fitted_model.feature_importances_
                # Determine feature names based on the model (ml_l includes treatment_col)
                if nuisance_key == 'ml_l':
                    # Features for ml_l are x_cols + [treatment_col]
                    feature_names = x_cols + [treatment_col]
                elif nuisance_key == 'ml_m':
                    # Features for ml_m are x_cols
                    feature_names = x_cols
                else:
                    continue # Should not happen with ml_l and ml_m

                # Ensure feature names match importance count
                if len(feature_names) != len(importances):
                    print(f"Warning: Feature names count ({len(feature_names)}) does not match importance count ({len(importances)}) for nuisance key '{nuisance_key}', rep {rep_idx}, fold {fold_idx}. Skipping.")
                    continue


                for feature_name, importance in zip(feature_names, importances):
                    records.append({
                        'nuisance': nuisance_key,
                        'rep': rep_idx,
                        'fold': fold_idx,
                        'feature': feature_name,
                        'importance': importance
                    })
    return pd.DataFrame(records)

# Now call the function for ml_l and ml_m
try:
    importance_ml_l_df = collect_importances('ml_l')
    if not importance_ml_l_df.empty:
        print("\nFeature Importances for ml_l (Outcome Model) across folds and repetitions:")
        print(importance_ml_l_df.groupby('feature')['importance'].mean().sort_values(ascending=False))
    else:
         print("\nNo feature importance data collected for ml_l.")
except Exception as e:
     print(f"\nAn error occurred while collecting importances for ml_l: {e}")


try:
    importance_ml_m_df = collect_importances('ml_m')
    if not importance_ml_m_df.empty:
        print("\nFeature Importances for ml_m (Treatment Model) across folds and repetitions:")
        print(importance_ml_m_df.groupby('feature')['importance'].mean().sort_values(ascending=False))
    else:
         print("\nNo feature importance data collected for ml_m.")
except Exception as e:
     print(f"\nAn error occurred while collecting importances for ml_m: {e}")

/usr/local/lib/python3.11/dist-packages/sklearn/utils/deprecation.py:151: FutureWarning: 'force_all_finite' was renamed to 'ensure_all_finite' in 1.6 and will be removed in 1.8.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/sklearn/utils/deprecation.py:151: FutureWarning: 'force_all_finite' was renamed to 'ensure_all_finite' in 1.6 and will be removed in 1.8.
  warnings.warn(



DoubleMLPLR main results
           coef   std err         t     P>|t|     2.5 %    97.5 %
mhtx -0.020792  0.007177 -2.896933  0.003768 -0.034859 -0.006725 

Full dml_plr.models structure:
{'ml_l': {'mhtx': [[GradientBoostingRegressor(max_depth=4, n_estimators=200, random_state=42), GradientBoostingRegressor(max_depth=4, n_estimators=200, random_state=42), GradientBoostingRegressor(max_depth=4, n_estimators=200, random_state=42), GradientBoostingRegressor(max_depth=4, n_estimators=200, random_state=42), GradientBoostingRegressor(max_depth=4, n_estimators=200, random_state=42)]]}, 'ml_m': {'mhtx': [[RandomForestRegressor(max_depth=4, n_estimators=200, random_state=42), RandomForestRegressor(max_depth=4, n_estimators=200, random_state=42), RandomForestRegressor(max_depth=4, n_estimators=200, random_state=42), RandomForestRegressor(max_depth=4, n_estimators=200, random_state=42), RandomForestRegressor(max_depth=4, n_estimators=200, random_state=42)]]}}
-----------------------------------

In [42]:
import pandas as pd
import numpy as np
from doubleml import DoubleMLData, DoubleMLPLR
from sklearn.ensemble import GradientBoostingRegressor, RandomForestRegressor

# Load your data (adjust path as needed)
file_path = '/content/Final Imputed dataset with dummies 7.7.25.xlsx'
df = pd.read_excel(file_path)

# Define variables using your actual column names
y_col = 'aggemp'
treatment_col = 'mhtx'
x_cols = [
    'uv_index', 'gfcf', 'rnd', 'internet', 'mobile', 'fdi',
    'lnwages', 'tariff', 'msch', 'reer', 'covid_dummy'
]

# Prepare data
required_cols = [y_col, treatment_col] + x_cols
df_plr = df[required_cols].dropna()

# Create DoubleMLData object
dml_data = DoubleMLData(df_plr, y_col=y_col, d_cols=treatment_col, x_cols=x_cols)

# Define ML learners
ml_l = GradientBoostingRegressor(n_estimators=200, max_depth=4, random_state=42)
ml_m = RandomForestRegressor(n_estimators=200, max_depth=4, random_state=42)

# Initialize DoubleMLPLR (do NOT include ml_g for 'partialling out' score)
dml_plr = DoubleMLPLR(dml_data, ml_l=ml_l, ml_m=ml_m)

# Fit the model with store_models=True to access fitted models
dml_plr.fit(store_models=True)

# Print main results
print("DoubleMLPLR Main Results Summary:")
print(dml_plr.summary)

# Access fitted models for feature importance
print("\n" + "="*60)
print("FEATURE IMPORTANCE ANALYSIS")
print("="*60)

# Check if models are stored
if dml_plr.models is not None:
    # Access fitted models for the first fold of the first repetition
    # Structure is dml_plr.models[nuisance_key][treatment_col][rep_idx][fold_idx]
    fitted_ml_l = dml_plr.models['ml_l'][treatment_col][0][0]
    fitted_ml_m = dml_plr.models['ml_m'][treatment_col][0][0]


    # Display feature importances for ml_l (outcome model)
    print("\nFeature importances for ml_l (outcome model - predicting Y|X):")
    if hasattr(fitted_ml_l, 'feature_importances_'):
        # Features for ml_l are x_cols + [treatment_col]
        all_features_l = x_cols + [treatment_col]
        if len(fitted_ml_l.feature_importances_) == len(all_features_l):
             feature_importances_l = pd.Series(fitted_ml_l.feature_importances_, index=all_features_l)
             print(feature_importances_l.sort_values(ascending=False))
        else:
            print(f"Feature importance count ({len(fitted_ml_l.feature_importances_)}) does not match expected feature count ({len(all_features_l)}). Cannot display with names.")
            print(fitted_ml_l.feature_importances_)

    else:
        print("The ml_l model does not have a feature_importances_ attribute.")

    # Display feature importances for ml_m (treatment model)
    print("\nFeature importances for ml_m (treatment model - predicting D|X):")
    if hasattr(fitted_ml_m, 'feature_importances_'):
         # Features for ml_m are x_cols
        if len(fitted_ml_m.feature_importances_) == len(x_cols):
            feature_importances_m = pd.Series(fitted_ml_m.feature_importances_, index=x_cols)
            print(feature_importances_m.sort_values(ascending=False))
        else:
            print(f"Feature importance count ({len(fitted_ml_m.feature_importances_)}) does not match expected feature count ({len(x_cols)}). Cannot display with names.")
            print(fitted_ml_m.feature_importances_)
    else:
        print("The ml_m model does not have a feature_importances_ attribute.")

    # Create a comparison DataFrame (only if both importances were successfully named)
    if (hasattr(fitted_ml_l, 'feature_importances_') and len(fitted_ml_l.feature_importances_) == len(all_features_l) and
        hasattr(fitted_ml_m, 'feature_importances_') and len(fitted_ml_m.feature_importances_) == len(x_cols)):

        print("\nFeature Importance Comparison (First Fold, First Repetition):")
        # Align features for comparison - this might be tricky if ml_l feature order is unexpected
        # Assuming x_cols order is consistent, we can compare importances for x_cols in both models
        comparison_df = pd.DataFrame({
            'Outcome_Model_Importance (ml_l, predicting Y|D,X)': feature_importances_l.drop(treatment_col, errors='ignore'), # Drop treatment from ml_l for X comparison
            'Treatment_Model_Importance (ml_m, predicting D|X)': feature_importances_m
        })
        comparison_df = comparison_df.sort_values('Outcome_Model_Importance (ml_l, predicting Y|D,X)', ascending=False)
        print(comparison_df.round(4))

else:
    print("Models were not stored. Make sure to use fit(store_models=True)")

# Additional analysis: Feature importance across all folds - This requires iterating through the structure.
# The collect_importances function from cell d6b21281 is designed for this.
# If you want to see average importances across all folds/reps, use that function.

/usr/local/lib/python3.11/dist-packages/sklearn/utils/deprecation.py:151: FutureWarning: 'force_all_finite' was renamed to 'ensure_all_finite' in 1.6 and will be removed in 1.8.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/sklearn/utils/deprecation.py:151: FutureWarning: 'force_all_finite' was renamed to 'ensure_all_finite' in 1.6 and will be removed in 1.8.
  warnings.warn(


DoubleMLPLR Main Results Summary:
          coef   std err         t     P>|t|    2.5 %    97.5 %
mhtx -0.018969  0.007899 -2.401627  0.016322 -0.03445 -0.003489

FEATURE IMPORTANCE ANALYSIS

Feature importances for ml_l (outcome model - predicting Y|X):
Feature importance count (11) does not match expected feature count (12). Cannot display with names.
[0.04097229 0.13943842 0.19390447 0.04411581 0.03638213 0.05347868
 0.1971535  0.07277085 0.17365881 0.04783531 0.00028973]

Feature importances for ml_m (treatment model - predicting D|X):
rnd            0.565561
lnwages        0.156516
tariff         0.069304
uv_index       0.056472
msch           0.047034
fdi            0.027531
gfcf           0.025926
mobile         0.022725
internet       0.019922
reer           0.008971
covid_dummy    0.000039
dtype: float64


In [45]:
from doubleml import DoubleMLData, DoubleMLPLR
from sklearn.ensemble import GradientBoostingRegressor, RandomForestRegressor

# Prepare your data as before
y_col = 'aggemp'
treatment_col = 'uv_index' # mhtx is a continuous treatment variable
x_cols = [
    'gfcf', 'rnd', 'internet', 'mobile', 'fdi',
    'lnwages', 'tariff', 'msch', 'reer', 'covid_dummy' # Removed 'uv_index'
]

# Drop missing values in required columns
df_plr = df[[y_col, treatment_col] + x_cols].dropna()

# Set up DoubleML Data for PLR
dml_data = DoubleMLData(df_plr, y_col=y_col, d_cols=treatment_col, x_cols=x_cols)

# Specify Machine Learning Models
ml_g = GradientBoostingRegressor(n_estimators=200, max_depth=4, random_state=42)
ml_m = RandomForestRegressor(n_estimators=200, max_depth=4, random_state=42)
ml_l = RandomForestRegressor(n_estimators=200, max_depth=4, random_state=42)

# Fit DoubleMLPLR
# Removed store_models=True from constructor
dml_plr = DoubleMLPLR(
    dml_data,
    ml_g=ml_g,
    ml_m=ml_m,
    ml_l=ml_l
)
# Added store_models=True to the fit method
dml_plr.fit(store_models=True)

# Print the summary of the results
print(dml_plr.summary)

/usr/local/lib/python3.11/dist-packages/doubleml/plm/plr.py:109: UserWarning: A learner ml_g has been provided for score = "partialling out" but will be ignored. "A learner ml_g is not required for estimation.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/sklearn/utils/deprecation.py:151: FutureWarning: 'force_all_finite' was renamed to 'ensure_all_finite' in 1.6 and will be removed in 1.8.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/sklearn/utils/deprecation.py:151: FutureWarning: 'force_all_finite' was renamed to 'ensure_all_finite' in 1.6 and will be removed in 1.8.
  warnings.warn(


              coef   std err         t     P>|t|     2.5 %    97.5 %
uv_index  0.006654  0.008699  0.764946  0.444304 -0.010395  0.023704


In [44]:
df.head()

,country,time,mhtx,rnd,tariff,mobile,uv_index,gfcf,fdi,lnwages,reer,d1,aggemp,ctryid,dmy_uvindex,dmy_mhtx,msch,internet,covid_dummy,L_aggemp
0,Argentina,1990,23.6244,0.472133,14.550,0.036767,40.5,13.9970,1.29888,22.4846,195.851,0,46.6,1,0.0,0.0,8.12256,0.436837,0,NaN
1,Argentina,1991,23.6244,0.472133,12.383,0.075516,50.2,14.6370,1.28558,22.7368,193.781,0,47.0,1,0.0,0.0,8.19000,0.246292,0,46.6
2,Argentina,1992,23.6244,0.450528,12.660,0.138792,52.4,16.7024,1.93679,22.6555,188.565,0,47.3,1,0.0,0.0,8.25800,0.326213,0,47.0
3,Argentina,1993,28.9722,0.441513,12.530,0.329148,52.9,19.6881,1.17980,22.7717,221.030,0,47.1,1,0.0,0.0,8.32600,0.259685,0,47.3
4,Argentina,1994,30.0064,0.436208,11.723,0.699252,57.3,19.9656,1.41195,22.8847,213.286,0,44.8,1,0.0,0.0,8.39400,0.236687,0,47.1


In [46]:
from doubleml import DoubleMLData, DoubleMLPLR
from sklearn.ensemble import GradientBoostingRegressor, RandomForestRegressor

# Prepare your data as before
y_col = 'aggemp'
treatment_col = 'uv_index' # mhtx is a continuous treatment variable
x_cols = [
    'gfcf', 'rnd', 'internet', 'mobile', 'fdi',
    'lnwages', 'tariff', 'msch', 'reer'
]

# Drop missing values in required columns
df_plr = df[[y_col, treatment_col] + x_cols].dropna()

# Set up DoubleML Data for PLR
dml_data = DoubleMLData(df_plr, y_col=y_col, d_cols=treatment_col, x_cols=x_cols)

# Specify Machine Learning Models
ml_g = GradientBoostingRegressor(n_estimators=200, max_depth=4, random_state=42)
ml_m = RandomForestRegressor(n_estimators=200, max_depth=4, random_state=42)
ml_l = RandomForestRegressor(n_estimators=200, max_depth=4, random_state=42) # Added ml_l

# Fit DoubleMLPLR
dml_plr = DoubleMLPLR(dml_data, ml_g=ml_g, ml_m=ml_m, ml_l=ml_l) # Added ml_l argument
# Added store_models=True to the fit method
dml_plr.fit(store_models=True)


# Print the summary of the results
print(dml_plr.summary)

/usr/local/lib/python3.11/dist-packages/doubleml/plm/plr.py:109: UserWarning: A learner ml_g has been provided for score = "partialling out" but will be ignored. "A learner ml_g is not required for estimation.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/sklearn/utils/deprecation.py:151: FutureWarning: 'force_all_finite' was renamed to 'ensure_all_finite' in 1.6 and will be removed in 1.8.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/sklearn/utils/deprecation.py:151: FutureWarning: 'force_all_finite' was renamed to 'ensure_all_finite' in 1.6 and will be removed in 1.8.
  warnings.warn(


              coef   std err         t   P>|t|     2.5 %    97.5 %
uv_index  0.008241  0.008498  0.969693  0.3322 -0.008416  0.024897


In [48]:
from doubleml import DoubleMLData, DoubleMLPLR
from sklearn.ensemble import GradientBoostingRegressor, RandomForestRegressor

# Prepare your data as before
y_col = 'aggemp'
treatment_col = 'uv_index' # mhtx is a continuous treatment variable
x_cols = [
    'gfcf', 'rnd', 'internet', 'mobile', 'fdi',
    'lnwages', 'tariff', 'msch', 'reer', 'covid_dummy', 'dmy_uvindex'
]

# Drop missing values in required columns
df_plr = df[[y_col, treatment_col] + x_cols].dropna()

# Set up DoubleML Data for PLR
dml_data = DoubleMLData(df_plr, y_col=y_col, d_cols=treatment_col, x_cols=x_cols)

# Specify Machine Learning Models
ml_g = GradientBoostingRegressor(n_estimators=200, max_depth=4, random_state=42)
ml_m = RandomForestRegressor(n_estimators=200, max_depth=4, random_state=42)
ml_l = RandomForestRegressor(n_estimators=200, max_depth=4, random_state=42) # Added ml_l

# Fit DoubleMLPLR
dml_plr = DoubleMLPLR(dml_data, ml_g=ml_g, ml_m=ml_m, ml_l=ml_l) # Added ml_l argument
# Added store_models=True to the fit method
dml_plr.fit(store_models=True)


# Print the summary of the results
print(dml_plr.summary)

/usr/local/lib/python3.11/dist-packages/doubleml/plm/plr.py:109: UserWarning: A learner ml_g has been provided for score = "partialling out" but will be ignored. "A learner ml_g is not required for estimation.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/sklearn/utils/deprecation.py:151: FutureWarning: 'force_all_finite' was renamed to 'ensure_all_finite' in 1.6 and will be removed in 1.8.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/sklearn/utils/deprecation.py:151: FutureWarning: 'force_all_finite' was renamed to 'ensure_all_finite' in 1.6 and will be removed in 1.8.
  warnings.warn(


              coef   std err         t     P>|t|     2.5 %    97.5 %
uv_index  0.013048  0.010825  1.205318  0.228081 -0.008169  0.034264


In [49]:
from doubleml import DoubleMLData, DoubleMLPLR
from sklearn.ensemble import GradientBoostingRegressor, RandomForestRegressor

# Prepare your data as before
y_col = 'aggemp'
treatment_col = 'uv_index' # mhtx is a continuous treatment variable
x_cols = [
    'gfcf', 'rnd', 'internet', 'mobile', 'fdi',
    'lnwages', 'tariff', 'msch', 'reer', 'dmy_uvindex'
]

# Drop missing values in required columns
df_plr = df[[y_col, treatment_col] + x_cols].dropna()

# Set up DoubleML Data for PLR
dml_data = DoubleMLData(df_plr, y_col=y_col, d_cols=treatment_col, x_cols=x_cols)

# Specify Machine Learning Models
ml_g = GradientBoostingRegressor(n_estimators=200, max_depth=4, random_state=42)
ml_m = RandomForestRegressor(n_estimators=200, max_depth=4, random_state=42)
ml_l = RandomForestRegressor(n_estimators=200, max_depth=4, random_state=42) # Added ml_l

# Fit DoubleMLPLR
dml_plr = DoubleMLPLR(dml_data, ml_g=ml_g, ml_m=ml_m, ml_l=ml_l) # Added ml_l argument
# Added store_models=True to the fit method
dml_plr.fit(store_models=True)


# Print the summary of the results
print(dml_plr.summary)

/usr/local/lib/python3.11/dist-packages/doubleml/plm/plr.py:109: UserWarning: A learner ml_g has been provided for score = "partialling out" but will be ignored. "A learner ml_g is not required for estimation.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/sklearn/utils/deprecation.py:151: FutureWarning: 'force_all_finite' was renamed to 'ensure_all_finite' in 1.6 and will be removed in 1.8.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/sklearn/utils/deprecation.py:151: FutureWarning: 'force_all_finite' was renamed to 'ensure_all_finite' in 1.6 and will be removed in 1.8.
  warnings.warn(


              coef   std err         t     P>|t|     2.5 %   97.5 %
uv_index  0.012396  0.010298  1.203732  0.228693 -0.007788  0.03258
